<a href="https://colab.research.google.com/github/laasyagudisa/ML_Assignments_Data/blob/main/NeuralNetworkModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
#pip install ucimlrepo

##Import Data

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets


df = pd.concat([X,y],axis=1)

##Define NeuralNetwork Class
* Class definition includes activation, learning_rate, input_size, hidden_layer_size, and output_layer_size
* Methods defined include forward pass and back-propagation, using *Stochastic Gradient Descent*

In [ ]:
class NNClassifier:
    def __init__(self, activation, learn_rate, input_neurons, hidden_neurons=2, output_neurons=1):
        self.activation = activation
        self.learn_rate = learn_rate
        self.input_neurons = input_neurons
        self.hidden_neurons = hidden_neurons
        self.output_neurons = output_neurons

        self.input_weights = np.random.randn(input_neurons, hidden_neurons)
        self.hidden_weights = np.random.randn(hidden_neurons, output_neurons)
        self.hidden_bias = np.zeros((1, hidden_neurons))
        self.output_bias = np.zeros((1, output_neurons))

    @staticmethod
    def pre_process(mydata):
        newdata = mydata.copy()
        newdata = newdata.drop_duplicates()
        newdata = newdata.dropna()
        label_encoder = LabelEncoder()
        for column in newdata.columns:
            if newdata[column].dtype == 'object':
                newdata[column] = label_encoder.fit_transform(mydata[column])
        return newdata

    @staticmethod
    def split(X, Y):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)
        return X_train, X_test, Y_train, Y_test


    def forward_act(self, X):
        if self.activation == 'sigmoid':
            X = np.clip(X,-500,500) #stops overflow and underflow errors
            return 1 / (1 + np.exp(-X)) #sigmoid function
        elif self.activation == 'tanh':
            return np.tanh(X)  #tanh function
        else:
            return np.maximum(0, X) #relu function

    def backward_act(self, X):
        if self.activation == 'sigmoid':
            return X * (1 - X)  # Derivative of sigmoid
        elif self.activation == 'tanh':
            return 1 - X ** 2  # Derivative of tanh
        else:
            return np.where(X > 0, 1, 0)  # Derivative of ReLU

    def forward(self, X):
        self.hidden = self.forward_act(np.dot(X, self.input_weights) + self.hidden_bias)
        self.output = self.forward_act(np.dot(self.hidden, self.hidden_weights) + self.output_bias)
        return np.round(self.output)

    def backward_prop(self, X, Y):
        error = Y - np.round(self.output)
        delta_output = error * self.backward_act(self.output)
        error_hidden = delta_output.dot(self.hidden_weights.T)
        delta_hidden = error_hidden * self.backward_act(self.hidden)

        # Update weights and biases using SGD
        self.hidden_weights += self.hidden.T.dot(delta_output) * self.learn_rate
        self.input_weights += X.T.dot(delta_hidden) * self.learn_rate
        self.output_bias += np.sum(delta_output, axis=0, keepdims=True) * self.learn_rate
        self.hidden_bias += np.sum(delta_hidden, axis=0, keepdims=True) * self.learn_rate

    def training(self, X, Y, num_iter):
        for i in range(num_iter):
            for x, y in zip(X, Y):
                x = x.reshape(1, -1)
                y = y.reshape(1, -1)
                #Forward and backward pass for each data point
                self.forward(x)
                self.backward_prop(x, y)

    def predict(self, X):
        return self.forward(X)

In [ ]:
df = NNClassifier.pre_process(df)
y = df['Diagnosis']
X = df.drop('Diagnosis', axis=1)

##Data Preparation
* Clean input data *(data cleaner built into NN Class)*
* Split train/test sets

In [ ]:
X_train, X_test, Y_train, Y_test = NNClassifier.split(X, y)

##Create and Train NN Classifier Model

In [ ]:
nn = NNClassifier('signmoid', 0.001, 30, 15, 1)
nn.training(X_train.values, Y_train.values, 1000)

##Check Model Accuracy

In [ ]:
Y_pred = np.round(nn.predict(X_train))
train_accuracy = accuracy_score(Y_train, Y_pred)
test_accuracy = accuracy_score(Y_test, np.round(nn.predict(X_test)))
print("The training and testing accuracy : ")
print(f'\tTraining Accuracy: {train_accuracy:.5f}')
print(f'\tTesting  Accuracy: {test_accuracy:.5f}')

The training and testing accuracy : 
	Training Accuracy: 0.63956
	Testing  Accuracy: 0.57895
